# BCDP CMIP6 Example
## Quick Overview
Here we will do a quick walkthrough of using BCDP to process some regional climate simulations from the Coordinate Regional Downscaling Experiment

In [1]:
import os
import glob
import numpy as np
import bcdp
import matplotlib.pyplot as plt

Create a file conventions template. Here we only need the model and variable names so the rest of the file template is filled in by wildcards.

In [7]:
# Dataset Options
dataset = 's3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/'

# Processing Options
backend = 'scipy'
method = 'linear'
res = (0.88, 0.88)
freq = 'A'
lat_bnds = None
lon_bnds = None
time_bnds = None
season = None

# Output
nc_file = 'CMIP_example.nc' # type: stage-out
plot_file = 'CMIP_example.png' # type: stage-out

Load the data. Because we have loaded the template, the loader now knows exactly how to extract the required informations from the filenames.

In [35]:
import pandas as pd
ens = bcdp.load_bucket(dataset, anon=True)

/Users/goodman/opt/envs/py37/lib/python3.7/site-packages/xarray/core/dataarray.py:3268: FutureWarning: roll_coords will be set to False in the future. Explicitly set roll_coords to silence warning.
  shifts=shifts, roll_coords=roll_coords, **shifts_kwargs


The loader returns an `Ensemble` object, which is essentially a collection of datasets and applies preprocessing operations to each of them. Here we will regrid the data to a coarser (0.88 degree) grid using bilinear interpolation, and consider only the winter months (DJF).

In [36]:
output_grid = bcdp.utils.grid_from_res(res, ens.overlap)
ens_u = ens.homogenize(backend=backend, method=method, freq=freq,
                       output_grid=output_grid, clean=False)

Now that the underlying datastructures are homogeneous (same grid and time step), we can convert it to an xarray dataarray which has dimensions (names, time, lat, lon).

In [37]:
def show_ens(ens, nc_file, plot_file):
    #da = ens.bundle('Temperature').add_mean('CMIP6').first
    da = ens.bundle('Temperature').first
    da.to_netcdf(nc_file)
    plt.figure()
    da.mean('time').plot(x='x', y='y', col='names', col_wrap=3)
    plt.savefig(plot_file)

We can easily visualize the annual climatology with xarray's built-in plotting methods.

In [39]:
ens_u.bundle('Temperature').first

,Array,Chunk
Bytes,52.65 MiB,326.72 kiB
Shape,"(1, 165, 205, 408)","(1, 1, 205, 408)"
Count,1522 Tasks,165 Chunks
Type,float32,numpy.ndarray
